In [7]:
import pyspark
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("Classification").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


In [22]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [9]:

# import os
# import boto3
# import gzip
# s3 = boto3.resource('s3')
# PATH = "s3://act-pyspark/949d9f4e6cfcc02b1c7fd2ef726be770/31988545f6a0cacac3cf4a5090e106d5/health/Core/Health/Assertive Community Treatment Outcomes/data.csv.gz"

In [10]:
# mlflow.set_experiment(experiment_name = "sd_ml")

# #set up client
# from mlflow.tracking import MlflowClient
# client = MlflowClient()

In [4]:
# #create a run

# # Create a run and attach it to the experiment you just created
# experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment

# experiment_name = "sd-ml"
# def create_run(experiment_name):
#     mlflow.set_experiment(experiment_name = experiment_name)
#     for x in experiments:
#         if experiment_name in x.name:
# #             print(experiment_name)
# #             print(x)
#             experiment_index = experiments.index(x)
#             run = client.create_run(experiments[experiment_index].experiment_id) # returns mlflow.entities.Run
# #             print(run)
#             return run

# # Example run command
# # run = create_run('Experiment-3')
# # run = create_run(experiment_name)
# # add tags to run
# # add params and metrics to a run
# # #terminate client

In [5]:
# # test the functionality here
# run = create_run('sd-ml')

# # Add tag to a run
# client.set_tag(run.info.run_id, "Algorithm", "Gradient Boosted Tree")
# client.set_tag(run.info.run_id,"Random Seed",999)
# client.set_tag(run.info.run_id,"Train Perct",999)

# # Add params and metrics to a run
# client.log_param(run.info.run_id, "Max Depth", 999)
# client.log_param(run.info.run_id, "Max Bins", 999)
# client.log_metric(run.info.run_id, "Accuracy", 999)

# # Terminate the client
# client.set_terminated(run.info.run_id)

INFO: 'sd-ml' does not exist. Creating a new experiment


AttributeError: 'NoneType' object has no attribute 'info'

In [9]:

# obj = s3.Object("act-pyspark", 
#                 "949d9f4e6cfcc02b1c7fd2ef726be770/31988545f6a0cacac3cf4a5090e106d5/health/Core/Health/Assertive Community Treatment Outcomes/data.csv.gz")
# # with gzip.GzipFile(fileobj=obj.get()["Body"]) as gzipfile:
# #     content = gzipfile.read()
# #     print(content)

# act_df= spark.read.csv(PATH, inferSchema = True, header = True)
# act_df.limit(6).toPandas()

In [35]:
schema = StructType([
    StructField("age", FloatType(), True),
    StructField("sex", FloatType(), True),
    StructField("chest_pain_type", FloatType(), True),
    StructField("resting_bps", FloatType(), True),
    StructField("chol", FloatType(), True),
    StructField("fast_blood_sugar", FloatType(), True),
    StructField("rest_ecg_type", FloatType(), True),
    StructField("max_hr", FloatType(), True),
    StructField("exercise_angina", FloatType(), True),
    StructField("oldpeak", FloatType(), True),
    StructField("slope_type", FloatType(), True),
    StructField("colored_arteries", FloatType(), True),
    StructField("thal_type", FloatType(), True),
    StructField("heart_disease", IntegerType(), True),
])

df = spark.read.csv("data/processed.cleveland.data", schema = schema, header = False, nullValue='?')
df.limit(6).toPandas()

,age,sex,chest_pain_type,resting_bps,chol,fast_blood_sugar,rest_ecg_type,max_hr,exercise_angina,oldpeak,slope_type,colored_arteries,thal_type,heart_disease
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
5,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0


In [36]:
df.groupBy('heart_disease').count().show()

+-------------+-----+
|heart_disease|count|
+-------------+-----+
|            1|   55|
|            3|   35|
|            4|   13|
|            2|   36|
|            0|  164|
+-------------+-----+



In [37]:
#check how much missing data there is
data_agg = df.agg(*[count(when(isnull(c), c)).alias(c) for c in df.columns])
print(data_agg.limit(8).toPandas())

df_clean = df.na.drop()
features = df.columns[:-1]
label = 'heart_disease'

   age  sex  chest_pain_type  resting_bps  chol  fast_blood_sugar  \
0    0    0                0            0     0                 0   

   rest_ecg_type  max_hr  exercise_angina  oldpeak  slope_type  \
0              0       0                0        0           0   

   colored_arteries  thal_type  heart_disease  
0                 4          2              0  


In [27]:
# df_renamed = df.withColumn('label_str', df[label].cast(StringType()))
labelIndexer = StringIndexer(inputCol=label, outputCol = 'label')
indexed = labelIndexer.fit(df).transform(df)

In [28]:
indexed.select('label').show()

+-----+
|label|
+-----+
|  1.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  1.0|
|  0.0|
|  0.0|
|  0.0|
|  1.0|
|  0.0|
|  1.0|
|  0.0|
|  1.0|
|  0.0|
|  1.0|
|  0.0|
+-----+
only showing top 20 rows



In [29]:
# change string variables to numeric
def to_numeric(indexed, features):
    numeric_inputs = []
    str_inputs = []
    for column in features:
        if str(indexed.schema[column].dataType) == 'StringType':
            stringIndexer = StringIndexer(inputCol = column, outputCol = column+'_num')
            indexed = stringIndexer.fit(indexed).transform(indexed)
            new_col_name = column+'_num'
            str_inputs.append(new_col_name)
        else:
            numeric_inputs.append(column)
    return indexed, numeric_inputs, str_inputs

indexed, numeric_inputs, str_inputs = to_numeric(indexed, features)


In [30]:
def cap_and_transform__skewed_cols(indexed):
    d = {}
    for col in numeric_inputs:
        # get values corresponding to bottom and top 1% of distribution
        d[col] = indexed.approxQuantile(col, [0.01, 0.99], 0.25)

        # transform col if skewed
        skew = indexed.agg(f.skewness(indexed[col])).collect()
        print(skew)
        skew = skew[0][0]
        if skew > 1:
            #cap at top and bottom
            indexed = indexed.withColumn(col, \
                                        log(when(df[col] < d[col][0], d[col][0])\
                                           .when(indexed[col] > d[col][1], d[col][1])\
                                           .otherwise(indexed[col]) + 1).alias(col))
        elif skew < -1:
            #cap at top and bottom
            indexed = indexed.withColumn(col, \
                                        exp(when(df[col] < d[col][0], d[col][0])\
                                           .when(indexed[col] > d[col][1], d[col][1])\
                                           .otherwise(indexed[col]) + 1).alias(col))
    return indexed

indexed = cap_and_transform__skewed_cols(indexed)

[Row(skewness(A1)=-0.2563496093947895)]
[Row(skewness(A2)=0.20601797730122015)]
[Row(skewness(A3)=0.40260435804137373)]
[Row(skewness(A4)=-0.04935088083712881)]
[Row(skewness(A5)=-0.09879203126218447)]
[Row(skewness(A6)=-0.3110969802658277)]
[Row(skewness(A7)=-0.6285337660559676)]
[Row(skewness(A8)=0.1637338005140266)]
[Row(skewness(A9)=0.041754827339823934)]
[Row(skewness(A10)=-0.3506177559444581)]
[Row(skewness(Age_Mons)=-0.6341451203248881)]
[Row(skewness(Qchat-10-Score)=-0.08006349388828135)]


In [31]:
#what's minimum value in df?
minimums = df.select([min(c).alias(c) for c in df.columns if c in numeric_inputs])
min_array = minimums.select(array(numeric_inputs).alias("mins"))
df_minimum = min_array.select(array_min(min_array.mins)).collect()
df_minimum[0][0]

0

In [32]:
#create dense feature vector
features_list = numeric_inputs + str_inputs
assembler = VectorAssembler(inputCols = features_list, outputCol = 'features')
scaler = MinMaxScaler(inputCol="features", outputCol="features_scaled") 
pipeline = Pipeline(stages=[assembler, scaler])
scalerModel = pipeline.fit(indexed)
scaledData = scalerModel.transform(indexed).select('features', 'label')
scaledData.limit(10).toPandas()

,features,label
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...",1.0
1,"(1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",0.0
2,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...",0.0
3,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0
4,"[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0
5,"[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0
6,"(1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...",0.0
7,"(0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.0
8,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",1.0
9,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...",0.0


In [33]:
train, test = scaledData.randomSplit([0.7, 0.3])
print(f"train len: {train.count()}, test len: {test.count()}")

train len: 721, test len: 333


In [34]:
classifiers = [LogisticRegression(), GBTClassifier()]

for classifier in classifiers:
    print(classifier)
    BC_evaluator = BinaryClassificationEvaluator() 

    paramGrid = (ParamGridBuilder().addGrid(classifier.maxIter, [10,15,20]).build())

    crossval = CrossValidator(
        estimator = classifier,
        estimatorParamMaps = paramGrid,
        evaluator=BC_evaluator,
        numFolds= 2)

    fitModel = crossval.fit(train)

    best_model = fitModel.bestModel
    
# Linear regression:    
#     print(f'Intercept: {str(best_model.interceptVector)}')
#     print(f'Coefficients: {str(best_model.coefficientMatrix)}')

    predictions = fitModel.transform(test) #fitModel automatically uses best model
    area_under_roc = BC_evaluator.evaluate(predictions)
    print(f"Area under ROC: {area_under_roc}")


LogisticRegression_5b87d61772e0
Area under ROC: 1.0
GBTClassifier_f6620ded66d5
Area under ROC: 1.0


In [35]:
#classification diagnostics
trainingSummary = best_model.summary

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()

print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

# Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
classifier.setThreshold(bestThreshold)

AttributeError: 'GBTClassificationModel' object has no attribute 'summary'